<a href="https://colab.research.google.com/github/totvslabs/carol-notebooks/blob/main/notebooks/AccessAllTenants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Grant Access to All Tenants Within an Organization
`This notebook will grant you access to all Tenants within an Organization.`
`The script will ask for the following json on execution:`

```python
{
    "authentication_config" : {
        "username": "username@totvs.com.br",
        "password": "password",
        "organization": "YourOrganization",
        "tenantName": "YourTenantName"
    },
    "script_config" : {
        "role": "userRole",
        "emails": [
          "username@totvs.com.br",
          "username@totvs.com.br",
          "username@totvs.com.br"
        ]
    }
}
```

#### REQUIREMENTS
`These are the packages the script needs before execution.`

In [ ]:
%%capture
!pip install --quiet python-dotenv
!pip install --quiet curlify
!pip install --quiet git+https://github.com/totvslabs/pyCarol.git
!pip install --quiet pycarol "pyarrow<1"
!pip install --quiet git+https://github.com/jnefoussi/pytechfin.git

from pycarol import Carol, ApiKeyAuth, PwdAuth, Staging, DataModel
from pycarol import Storage, Subscription
import pandas as pd
import numpy as np
from pycarol.utils.miscellaneous import drop_duplicated_parquet
import json
from google.colab import files

#### CAROL LOGIN FUNCTIONS
`These are the functions made to login into Carol. They will be the same for all notebooks (ideally) and will use pyCarol.` 

[pyCarol reference](https://github.com/totvslabs/pyCarol)

In [ ]:
def carol_connect(username, password, organization, tenantName):
    print(f"Connecting to Carol tenant {tenantName}... ", end="\n")

    return Carol(domain=tenantName,
                auth=PwdAuth(username, password), organization=organization)

#### SCRIPT FUNCTIONS
`If the script requires more functions to execute, they will be here.`

#### CONFIGURATION FILE
`Now you will need to upload the configuration file with the format given above.`

In [ ]:
try:
    config_file = files.upload()
    config_json = json.loads(config_file[next(iter(config_file))].decode("utf-8"))
except:
    with open('./carol.json') as config_file:
        config_json = json.loads(config_file.read())
    config_file.close()

Saving carol.json to carol (7).json


#### SCRIPT EXECUTION
`The main execution of the script will happen here.`

In [ ]:
Carol = carol_connect(
    config_json['authentication_config']['username'], 
    config_json['authentication_config']['password'], 
    config_json['authentication_config']['organization'], 
    config_json['authentication_config']['tenantName'])

# Add user into the tenants
Carol.switch_org_level()
org_id = Carol._current_org()['mdmId']
all_tenants = Carol.call_api(f'v1/organizations/{org_id}/allTenants', 'GET')
for tenant in all_tenants:
    role = config_json['script_config']['role']
    emails = config_json['script_config']['emails']
    for email in emails:
        tenant_name = tenant['mdmName']
        tenant_id = tenant['mdmId']
        data = {tenant_name:[{"mdmUserLogin":email,"mdmRoleNames":[role]}]}
        q = {
        "mustList": [
            {
            "mdmFilterType": "TERM_FILTER",
            "mdmKey": "mdmUserLogin.raw",
            "mdmValue": email
            }
        ],
        "shouldList": [
        ]
        }
        extra_headers = {'overriddentenantid': tenant_id} 
        params = dict(   
            indexType='CONFIG', 
            scrollable=False,
            pageSize=-1,
        )
        result = Carol.call_api("v1/users/filterUsers", 'POST', data = q,  params=params, extra_headers=extra_headers )
        if len(result['hits'])==0:
            print(f'adding {email} to {tenant_name}')
            Carol.call_api('v1/organizations/assignUsersToEnvironments', 'POST', data = data )

Connecting to Carol tenant brenopapaunif... 
adding breno.zipoli@totvs.com.br to deteam
adding breno.zipoli@totvs.com.br to rui
adding breno.zipoli@totvs.com.br to sentencesimilarity
adding breno.zipoli@totvs.com.br to buzzi
adding breno.zipoli@totvs.com.br to pilotomastercrm
adding breno.zipoli@totvs.com.br to sabino
adding breno.zipoli@totvs.com.br to sandro
adding breno.zipoli@totvs.com.br to manualapps
adding breno.zipoli@totvs.com.br to mltutorial
adding breno.zipoli@totvs.com.br to rddemo
adding breno.zipoli@totvs.com.br to phildev
adding breno.zipoli@totvs.com.br to temphamilton
adding breno.zipoli@totvs.com.br to espacolatitude
adding breno.zipoli@totvs.com.br to testclockinwebclient
adding breno.zipoli@totvs.com.br to fernando
adding breno.zipoli@totvs.com.br to deassistant2
adding breno.zipoli@totvs.com.br to clockinportaria
adding breno.zipoli@totvs.com.br to dasc904
adding breno.zipoli@totvs.com.br to danilomota
adding breno.zipoli@totvs.com.br to hamilton
adding breno.zipo